#### Test mynds packages in the sandbox

In [ ]:
import ast

from pathlib import Path
from typing import Any, Callable, Optional

import h5py as h5
import numpy as np
import polars as pl

from mynd.camera import Camera, Metadata
from mynd.collections import CameraGroup
from mynd.io import read_data_frame
from mynd.tasks.ingestion import map_metadata_to_cameras

from mynd.utils.log import logger
from mynd.utils.time import get_time_string
from mynd.utils.result import Ok, Err, Result

from mynd.io.h5 import H5Database, create_file_database
from mynd.io.h5.camera_writers import (
    insert_camera_metadata_into,
    buffer_camera_metadata,
)


CameraID = Camera.Identifier
GroupID = CameraGroup.Identifier


def create_metadata_mockup(
    data_frame: pl.DataFrame,
) -> dict[CameraID, Metadata]:
    """Create a bunch of camera metadata."""

    camera_fields: dict[str, Metadata] = map_metadata_to_cameras(
        data_frame,
        label_column="label",
        data_columns=["exposure", "exposure_logged", "timestamp", "invalid"],
    )

    camera_metadata: dict[CameraID, Metadata] = dict()
    for index, (label, fields) in enumerate(camera_fields.items()):
        identifier: CameraID = CameraID(index, label)
        camera_metadata[identifier] = fields

    return camera_metadata


def main() -> None:
    """Main entrypoint."""

    DATETIME: str = get_time_string()
    DEVICE: Path = Path("/home/martin/data")
    SOURCE: Path = DEVICE / Path(
        "acfr_cameras_metadata/r23685bc_20100605_021022_camera_metadata.csv"
    )
    DESTINATION: Path = DEVICE / Path(
        f"acfr_cameras_metadata_dev/{DATETIME}_r23685bc_20100605_021022_metadata.h5"
    )

    PATHS: dict[str, Path] = {
        "input": SOURCE,
        "destination": DESTINATION,
    }

    match create_file_database(PATHS.get("destination")):
        case Ok(database):
            pass
        case Err(message):
            logger.error(message)
            return

    storage: H5Database.Group = database.create_group("metadata").unwrap()

    # Options
    save: bool = False
    identifier: GroupID = GroupID(key=0, label="r23685bc_20100605_021022")

    data_frame: pl.DataFrame = read_data_frame(PATHS.get("input")).unwrap()

    camera_metadata: dict[CameraID, Metadata] = create_metadata_mockup(
        data_frame
    )

    # TODO: Write camera metadata to storage
    write_result: Result = insert_camera_metadata_into(storage, camera_metadata)

    database.root.visit(logger.info)
    invalid: Optional[H5Database.Dataset] = database.get("metadata/invalid")

    if invalid is not None:
        logger.info(invalid)


# INVOKE MAIN
main()